In [1]:
import sys
import time
import datetime
import os
import tensorflow as tf
import argparse
import json
import socket
from PIL import Image
from util import *
import numpy as np
%load_ext autoreload
%autoreload 2
# %matplotlib notebook
from copy import deepcopy
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from matplotlib.colors import hsv_to_rgb
from helpers import *

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
logPath = "TRAINING/split/train/"
snapshotPath = "SNAPSHOTS/snapshots/"

In [3]:
with open("hyperParams.json") as f:
    instanceParams = json.load(f)

# instanceParams

In [4]:
printFrequency = instanceParams["printFreq"]
batchSize = instanceParams["batchSize"]

iterations = instanceParams["iterations"]
baseLearningRate = instanceParams["baseLR"]
learningRate = baseLearningRate
snapshotFrequency = instanceParams["snapshotFreq"]

from dotmap import DotMap
arg = DotMap()
arg.logDev = False
arg.resume = 'n'
resume, startIteration, snapshotFiles = checkResume(snapshotPath,logPath, arg)

No snapshots found, training from scratch


In [5]:
with tf.device("/gpu:0"):
    trainingData = TrainingData(batchSize,instanceParams)

Reading Tensor("ImagePairData/image_data_reader/Placeholder:0", dtype=string, device=/device:GPU:0)
Reading Tensor("ImagePairData/image_data_reader_1/Placeholder:0", dtype=string, device=/device:GPU:0)
Reading Tensor("ImagePairData/image_data_reader_2/Placeholder:0", dtype=string, device=/device:GPU:0)
Reading Tensor("ImagePairData/image_data_reader_3/Placeholder:0", dtype=string, device=/device:GPU:0)


Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [6]:
with tf.device("/gpu:0"):
    flow_gd = tf.Variable(tf.zeros([2, 448, 832, 2]), name = "my_new_var")
    photoAlpha = instanceParams["photoParams"]["robustness"]
    photoBeta = instanceParams["photoParams"]["scale"]

In [7]:
with tf.device("/gpu:0"):
    # init
    with tf.variable_scope("netShare"):
        networkBodyF = NetworkBody(trainingData,instanceParams)
    with tf.variable_scope("netShare",reuse=True):
        networkBodyB = NetworkBody(trainingData,instanceParams,flipInput=True)

    trainingLoss = TrainingLoss(instanceParams,networkBodyF,networkBodyB,trainingData, flow_gd)
    solver, solver_op, learningRateTensor = attachSolver(trainingLoss.loss)
    flowFinal = networkBodyF.flows[0]
    # loss scheduling
    recLossBWeightTensor = trainingLoss.recLossBWeight
    golTensor = trainingLoss.gol

Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [8]:
with tf.device("/gpu:0"):
    gamma = trainingLoss.gol * instanceParams["smoothParams"]["weight"]
    sd_lr = 1.0

In [9]:
with tf.device("/gpu:0"):
    ploss = photoLoss(flow_gd,
                      trainingData.frame0["rgbNorm"],
                      trainingData.frame1["rgbNorm"],
                      photoAlpha, photoBeta)
    closs = tf.square(flow_gd - tf.stop_gradient(flowFinal))
    closs = tf.reduce_sum(closs * gamma, axis=3, keep_dims=True)
    tf.summary.image("fhat", flowToRgb1(flow_gd))
    tf.summary.scalar("fhat_ploss", tf.reduce_mean(ploss))
    tf.summary.scalar("gamma", tf.reduce_mean(gamma))

    with tf.variable_scope(None, default_name="new_var_solver"):
        split_solver = tf.train.AdamOptimizer(learning_rate=sd_lr, beta1=0.9, beta2=0.999)
        split_solver_op = split_solver.minimize(ploss + closs)

In [10]:
variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

var_subset = []
uninit = []
for v in variables:
    if not ("new_var" in v.name):
        var_subset.append(v)
    else:
        uninit.append(v)

In [11]:
# merge summaries
merged = tf.summary.merge_all()
saver = tf.train.Saver(var_subset, max_to_keep=0)

In [12]:
printFrequency = 100
snapshotFrequency = 2000
iterations = 60000
startIteration = 0
print(printFrequency)
print(snapshotFrequency)
print(iterations)
print(startIteration)

100
2000
60000
0


In [13]:
# golSchedule = 10 * (1 - np.exp(-np.arange(0, 20000) / 5000.0)) # np.flip(10 - np.logspace(-4, 1, 1e4))
# plt.plot(golSchedule)

In [14]:
def golSchedule(prev=None, flow=None, fhat=None, lr=None):
    ret = prev + lr * np.square(flow - fhat)
    return ret

In [ ]:
fs = []
fhats = []
with sessionSetup(arg) as sess:
    sess.run(tf.initialize_all_variables())
    # sess.run(tf.initialize_variables(uninit))
    # saver.restore(sess, '../model_download_scripts/photometric_smoothness/weights/iter_0000000000500000.ckpt')
    trainingData.dataQueuer.start_queueing(sess)

    #start summary writer
    summary_writer = tf.summary.FileWriter(logPath, sess.graph)

    #run
    lastPrint = time.time()
    for i in range(startIteration, startIteration + iterations + 10):
        # scheduled values
        learningRate = learningRateSchedule(baseLearningRate, i)
        recLossBWeight = unsupLossBSchedule(i)

         #run training
        feed_dict = {
            learningRateTensor: 1e-5,
            golTensor: np.ones((2, 448, 832, 2)) * 10.0 # if i == 0 else golSchedule(gol, f1, f2, 1e-3)
        }
        summary,result,totalLoss, f1, nfg, pl, f2, gol = sess.run([
            merged,solver_op,trainingLoss.loss, flowFinal, split_solver_op,
            ploss, flow_gd, golTensor], feed_dict=feed_dict)

        if (i+1) % printFrequency == 0:
            timeDiff = time.time() - lastPrint
            itPerSec = printFrequency/timeDiff
            remainingIt = startIteration + iterations + 10 - i
            eta = remainingIt/itPerSec
            print("Iteration "+str(i+1)+": loss: "+str(totalLoss)+", iterations per second: "+str(
                itPerSec)+", ETA: "+str(datetime.timedelta(seconds=eta)))+", lr: "+str(learningRate)

            summary_writer.add_summary(summary,i+1)
            summary_writer.flush()
            lastPrint = time.time()
            fs.append(deepcopy(f1))
            fhats.append(deepcopy(f2))

        if (i+1) % snapshotFrequency == 0:
            saver.save(sess,"snapshots/iter_"+str(i+1).zfill(16)+".ckpt")

        sys.stdout.flush()

    trainingData.dataQueuer.close(sess)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [31]:
np.mean(np.square(fs[-1] - fhats[-1]))

0.10396073

### Testing

In [ ]:
datasetRoot = '../example_data/'
frame0Path = '../example_data/datalists/train_im0.txt'
frame1Path = '../example_data/datalists/train_im1.txt'
gt0Path = '../example_data/datalists/train_gt0.txt'
desiredHeight = 480
desiredWidth = 854

In [ ]:
with open(frame0Path) as f:
    imagePairs0 = [datasetRoot+x[:-1] for x in f.readlines()]
with open(frame1Path) as f:
    imagePairs1 = [datasetRoot+x[:-1] for x in f.readlines()]
with open(gt0Path) as f:
    gtPairs1 = [datasetRoot+x[:-1] for x in f.readlines()]

In [ ]:
imagePairs0 = [item for item in imagePairs0 if 'bus' in item]
imagePairs1 = [item for item in imagePairs1 if 'bus' in item]
gtPairs1 = [item for item in gtPairs1 if 'bus' in item]

In [ ]:
testData = TestData(imagePairs0,imagePairs1,gtPairs1,1,desiredHeight,desiredWidth)

In [ ]:
with tf.device("/gpu:0"):
    with tf.variable_scope("netShare"):
        networkBody = NetworkBody(testData,instanceParams)
    flowFinal = networkBody.flows[0]

In [ ]:
flowViz = flowToRgb(flowFinal)
transformGrid = flowTransformGrid(flowFinal)
mean = tf.expand_dims(tf.expand_dims(tf.expand_dims([0.407871, 0.457525, 0.481094], 0), 0), 0)

In [ ]:
saver = tf.train.Saver()

# config tensorflow
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True

In [ ]:
test_length = 70
start = 0
result_dir = 'results'
iterations = 515000

flows = []
viz = []
gradients = []
ground_truths = []
arrs = []
with tf.Session(config=config) as sess:
    saver.restore(sess,"snapshots/iter_0000000000002000.ckpt")

    # run
    lastPrint = time.time()
    for i in range(start, start + min(test_length, len(imagePairs0))):
        feed_dict = {
            testData.im0File: imagePairs0[i],
            testData.im1File: imagePairs1[i],
            testData.gt0File: gtPairs1[i]
        }
        hsv, f = sess.run([flowViz, flowFinal],feed_dict=feed_dict)

        flows.append(deepcopy(f))
        h, w = 448, 532
        arr = np.maximum(np.minimum(np.asarray(hsv),1), 0)
        arr = np.squeeze(np.asarray(arr*255,np.uint8))
        arrs.append(deepcopy(arr[:h,:w,:]))
        im = Image.fromarray(arr[:h,:w,:])
        viz.append(deepcopy(im))
        # im.save("{}/{}.png".format(result_dir, str(i).zfill(3)))